## Things to do:
#http://aiddata.org/country-level-research-datasets
1. Country to Country: sankey diagram with (a) sector in the middle (b) umbrella in the middle
2. *Liars*:  disbursed - committed (color shows # of failures): square tree 
3. *Corrupts*: total project cost- disbursed (color shows # of failures): square tree [do it by filtering for projects that were completed, i.e. start_date and end_date not zero]
4. **Association rule mining** on categorical variables associated with Nepal. (Present as word cloud with importance based on lift)
5. Pick some variables and predict whether a project is completed or not. rank the variables by importance. 

In [92]:
#load aid data, downloaded from http://aiddata.org/country-level-research-datasets
import pandas as pd
df=pd.read_csv('/Users/sumangiri/Downloads/AidData_v3_0/AidData_v3_0.csv')

In [93]:
#get rid of nulls
df=df.fillna(0)
df.head(2)

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,ftc,repay_type,outstanding_amount_usd_nominal,interest_amount_usd_nominal,expert_commitment_amount_usd_nominal,export_credit_amount_usd_nominal,expert_extended_amount_usd_nominal,additional_info,source,source_detail
0,54886623,31549672,2007,Germany,DE,Europe,Other,Fed.Min.,1,Brazil,...,1,0,0,0,0,0,0,0,OECD,CRS_ONLINE_MAY2011
1,54886662,18924088,2003,Belgium,BE,Europe,DGCD ONG,DGCD,1,"Congo, Democratic Republic of",...,1,0,0,0,0,0,0,0,OECD,CRS_ONLINE_MAY2011


In [94]:
df.rename(columns={'aiddata_sector_name': 'sector'}, inplace=True)

In [95]:
#Change sector to easily digestible format
mapping={'General environmental protection':'Environment', 'Basic health':'Health',
       'Development aid/food security assistance':'Development',
       'Population policies/ programmes and reproductive health':'Health',
       'Basic education':'Education', 'Communications':'Communication',
       'Other social infrastructure and services':'Infrastructure', 'Other':'Other',
       'Government and civil society, general':'Government', 'Emergency response':'Emergency',
       'Unallocated/  unspecified':'Unspecified', 'Trade policy and regulations':'Trade',
       'Business and other services':'Business', 'General budget support':'Budget',
       'Conflict prevention and resolution, peace and security':'Security',
       'Transport and storage':'Transportation',
       'Support to non- governmental organisations (ngos) and government organizations':'NGOs',
       'Health, general':'Health', 'Banking and financial services':'Finance',
       'Administrative costs of donors':'Administrative', 'Post-secondary education':'Education',
       'Forestry':'Forestry', 'Secondary education':'Education', 'Agriculture':'Agriculture', 'Fishing':'Fishing',
       'Water supply and sanitation':'Water', 'Education, level unspecified':'Education',
       'Disaster prevention and preparedness':'Disaster',
       'Energy generation and supply':'Energy', 'Reconstruction relief':'Reconstruction', 'Tourism':'Tourism',
       'Women':'Women', 'Industry':'Industry', 'Action relating to debt':'Debt',
       'Mineral resources and mining':'Minerals', 'Construction':'Construction', 'Humanitarian aid':'Humanitarian',
       'Refugees in donor countries':'Refugees', 'Government and civil society':'Government',
       'Other commodity assistance':'Commodity', 'Agriculture, forestry, fishing':'Agriculture',
       'Health':'Health', 'Industry, mining, construction':'Industry', ' ':'Other', 'Education':'Education',
       'Economic infrastructure and services':'Infrastructure', 'Production sectors':'Production',
       'Social infrastructure and services':'Infrastructure'}

In [96]:
df.sector=df.sector.map(mapping)

In [97]:
mapping2={'World Bank - International Bank for Reconstruction and Development (IBRD)': 'World Bank (IBRD)',\
         'World Trade Organization (WTO) - International Trade Centre': 'World Bank (WTO)',\
          'World Bank - International Development Association (IDA)': 'World Bank (IDA)',\
          'World Bank - International Finance Corporation (IFC)': 'World Bank (IFC)'}
# df.donor=df.donor.map(mapping2)

In [98]:
df['donor'] = df['donor'].replace(mapping2)

In [100]:
df.loc[df.total_project_cost> 10e10]

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,ftc,repay_type,outstanding_amount_usd_nominal,interest_amount_usd_nominal,expert_commitment_amount_usd_nominal,export_credit_amount_usd_nominal,expert_extended_amount_usd_nominal,additional_info,source,source_detail
1244777,2445044,0,1996,Nordic Development Fund (NDF),0,Multilaterals,0,0,0,South Africa,...,0,0,0,0,0,0,0,private sector project,Annual Report,0


In [101]:
df=df[df.aiddata_id!=2445044] #appears to have flawed amount numbers

In [102]:
#sanity check
df.commitment_amount_usd_constant.sum()/1e12, \
df.received_amount_usd_nominal.sum()/1e12,\
df.total_project_cost.sum()/1e12

(7.042232705254, 0.011237852761466736, 4.549806002948007)

In [103]:
#Select features of interest
df1=df[['donor',\
'recipient',\
'sector',\
'commitment_amount_usd_constant',\
'received_amount_usd_nominal',\
'total_project_cost']]

In [234]:
df1.head(2)

,donor,recipient,sector,commitment_amount_usd_constant,received_amount_usd_nominal,total_project_cost
0,Germany,Brazil,Environment,8799,0,0
1,Belgium,"Congo, Democratic Republic of",Health,1864392,0,0


In [268]:
amt_feature='commitment_amount_usd_constant'

In [269]:
#get donor to sector
d2s=df1[['donor','sector',amt_feature]]\
.groupby(['donor','sector']).sum().reset_index().\
sort_values(amt_feature,ascending=[0])
d2s.columns=['source','target','weight']

In [270]:
#get sector to recipient
s2r=df1[['sector','recipient',amt_feature]]\
.groupby(['recipient','sector']).sum().reset_index().\
sort_values(amt_feature,ascending=[0])
s2r=s2r[['sector','recipient',amt_feature]]
s2r.columns=['source','target','weight']

### Top 5 donors to top 10 recipients through top 5 sectors
for cases with more than USD 100 million disbursed

In [271]:
#get top 5 sectors (Exclude other which is the number 1 sector)
sct1= list(df1[[amt_feature,'sector']].groupby(['sector']).sum()\
.reset_index().sort_values(amt_feature,ascending=[0])\
.sector[1:8])

In [272]:
#take donors and recipients only as long as they are in the sectors of interest
d2s_subset=d2s[d2s.target.isin(sct1)]
s2r_subset=s2r[s2r.source.isin(sct1)]

In [273]:
#how top 5 donors give aid
#how top 15 recipients receive aid
rcp1=s2r_subset.groupby('target').sum().reset_index().sort_values('weight',ascending=[0]).target[0:10]
dnr1=d2s_subset.groupby('source').sum().reset_index().sort_values('weight',ascending=[0]).source[0:5]

In [274]:
#take donors and recipients only as long as they are top 10 or top 5
d2s_subset1=d2s_subset[d2s_subset.source.isin(dnr1)]
s2r_subset1=s2r_subset[s2r_subset.target.isin(rcp1)]

In [275]:
df_final=pd.concat([d2s_subset1,s2r_subset1],0).sort_values('weight',ascending=[0])

In [276]:
nodes=pd.DataFrame(pd.concat([df_final.source,df_final.target],0).unique(), columns=['id'])
idx=range(nodes.shape[0])
dc=dict()
#create a dictionary to establish a map
for i in idx:
    dc[nodes.iloc[i].id]=i

In [277]:
df_final.source=df_final.source.map(dc)
df_final.target=df_final.target.map(dc)

In [278]:
#Get rid of data where aid is low
df_final=df_final[df_final.weight > 1e8]

In [279]:
#Convert to billion and save file
df_final.weight=df_final.weight*1.0/1e9
df_final.to_csv('links_2016.csv')
nodes.to_csv('nodes_2016.csv')

### Aid flow to Nepal (in million)
(for cases with more than 10 million USD committed. How top 20 donors give aid to Nepal and Srilanka over 6 major sectors)

In [280]:
country_list=['Nepal', 'Sri Lanka']
# Get the sectors when recipient is Nepal
df_nep_temp=pd.concat([d2s,s2r],0).sort_values('weight',ascending=[0])
s2r_nep=df_nep_temp[df_nep_temp.target.isin(country_list)]
s2r_nep=s2r_nep[s2r_nep.weight>1e7]

In [281]:
#get donor to sector for when recipient is Nepal
d2s_nep=df1[['donor','sector',amt_feature]]\
[df1.recipient.isin(country_list)]\
.groupby(['donor','sector']).sum().reset_index().\
sort_values(amt_feature,ascending=[0])
d2s_nep.columns=['source','target','weight']
d2s_nep=d2s_nep[d2s_nep.weight>1e7]

In [282]:
#get the top "n" sectors
sct_nep=pd.concat([d2s_nep[['target','weight']],s2r_nep[['source','weight']].\
                   rename(columns={'source': 'target'})],0).\
groupby('target').sum().reset_index().sort_values('weight',ascending=[0]).target[0:10]

In [283]:
d2s_nep_subset=d2s_nep[d2s_nep.target.isin(sct_nep)]
s2r_nep_subset=s2r_nep[s2r_nep.source.isin(sct_nep)]

In [284]:
#take top 15 donors only
dnr_nep=d2s_nep_subset.groupby('source').sum().reset_index().sort_values('weight',ascending=[0]).source[0:20]
d2s_nep_subset1=d2s_nep_subset[d2s_nep_subset.source.isin(dnr_nep)]

In [285]:
df_nep=pd.concat([d2s_nep_subset1,s2r_nep_subset],0).sort_values('weight',ascending=[0])

In [286]:
nodes_nep=pd.DataFrame(pd.concat([df_nep.source,df_nep.target],0).unique(), columns=['id'])
idx_nep=range(nodes_nep.shape[0])
dc_nep=dict()
#create a dictionary to establish a map
for i in idx_nep:
    dc_nep[nodes_nep.iloc[i].id]=i

In [287]:
df_nep.source=df_nep.source.map(dc_nep)
df_nep.target=df_nep.target.map(dc_nep)

In [288]:
df_nep=df_nep[df_nep.weight > 1e7]

In [289]:
#Convert to billion and save file
df_nep.weight=df_nep.weight*1.0/1e6
df_nep.to_csv('links_nep_2016.csv')
nodes_nep.to_csv('nodes_nep_2016.csv')

In [256]:
#sanity check (it should be zero if 1 million threshold on s2r_nep and d2s_nep is not applied)
#s2r_nep.weight.sum()-d2s_nep.weight.sum()

### Make classifier visual

In [303]:
class_id=range(200)

In [258]:
#simulate data
from random import randint
labels=['A','B','C','D','E']
act_class=map(lambda x:labels[randint(0,4)] ,class_id)
pred_class=map(lambda x:labels[randint(0,4)]+'1' ,class_id)

In [259]:
#create dataFrame
import pandas as pd
df_clf=pd.DataFrame([class_id,act_class,pred_class]).T
df_clf.columns=['id','actual','predicted']
df_clf_final=df_clf.groupby(['actual','predicted']).count().reset_index()
df_clf_final.columns=['source','target','weight']

In [260]:
nodes_clf=pd.DataFrame(pd.concat([df_clf_final.source,df_clf_final.target],0).unique(), columns=['id'])
idx_clf=range(nodes_clf.shape[0])
dc_clf=dict()
for i in idx_clf:
    dc_clf[nodes_clf.iloc[i].id]=i

In [261]:
df_clf_final.source=df_clf_final.source.map(dc_clf)
df_clf_final.target=df_clf_final.target.map(dc_clf)

In [262]:
nodes_clf=pd.DataFrame(labels+labels, columns=['id'])

In [263]:
df_clf_final.to_csv('links_clf.csv')
nodes_clf.to_csv('nodes_clf.csv')